In [1]:
import torch
torch.cuda.is_available()


True

In [2]:
!pip install -q transformers datasets scikit-learn accelerate


In [3]:
import numpy as np
import torch
import pandas as pd

from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score

SEED = 42
MODEL_NAME = "xlm-roberta-large"
MAX_LEN = 128

np.random.seed(SEED)
torch.manual_seed(SEED)


In [4]:
dataset = load_dataset("sepidmnorozy/Urdu_sentiment")

train_full = dataset["train"]
test_full  = dataset["test"]

print(train_full.shape, test_full.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/685 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/294 [00:00<?, ? examples/s]

(685, 2) (294, 2)


In [5]:
# Create 16-Shot (First 16 usable samples per class)
train_pd = train_full.to_pandas()

shot_0 = train_pd[train_pd["label"] == 0].head(16)
shot_1 = train_pd[train_pd["label"] == 1].head(16)

train_16_df = pd.concat([shot_0, shot_1]).reset_index(drop=True)

print(train_16_df["label"].value_counts())
print(train_16_df.shape)


label
0    16
1    16
Name: count, dtype: int64
(32, 2)


In [6]:
# Tokenization
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [7]:
# Convert datasets
train_16_ds = Dataset.from_pandas(train_16_df)
test_16_ds  = test_full

train_16_ds = train_16_ds.map(tokenize, batched=True, remove_columns=["text"])
test_16_ds  = test_16_ds.map(tokenize, batched=True, remove_columns=["text"])

train_16_ds.set_format("torch")
test_16_ds.set_format("torch")

print(train_16_ds[0])


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/294 [00:00<?, ? examples/s]

{'label': tensor(0), 'input_ids': tensor([     0,  25345,     12, 124492, 127292,    490, 120918,  46362,  10203,
         18189, 163154,  49957,    190,   6866,  43292,      2,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
      

In [9]:
# Metric (Accuracy only)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}


In [10]:
# Load Model (FULL Fine-Tuning)
model_16 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Training Arguments
training_args_16 = TrainingArguments(
    output_dir="sst2_xlmr_16shot",
    eval_strategy="epoch",
    save_strategy="no",              # avoid disk issues
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # REQUIRED
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=False,
    fp16=True,
    logging_steps=10,
    report_to="none",
    seed=SEED
)


In [12]:
# Trainer
trainer_16 = Trainer(
    model=model_16,
    args=training_args_16,
    train_dataset=train_16_ds,
    eval_dataset=test_16_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-4167553178.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_16 = Trainer(


In [13]:
# Train
trainer_16.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.702947,0.513605
2,No log,0.697967,0.513605
3,No log,0.693999,0.517007
4,No log,0.693999,0.517007
5,No log,0.693286,0.510204
6,No log,0.693286,0.510204
7,No log,0.693053,0.517007
8,No log,0.692845,0.500000
9,No log,0.696014,0.486395
10,0.675200,0.707870,0.500000


TrainOutput(global_step=20, training_loss=0.6350215911865235, metrics={'train_runtime': 44.245, 'train_samples_per_second': 14.465, 'train_steps_per_second': 0.452, 'total_flos': 149109018132480.0, 'train_loss': 0.6350215911865235, 'epoch': 20.0})

In [14]:
# Final Evaluation
results_16 = trainer_16.evaluate()
results_16


{'eval_loss': 0.7136549949645996,
 'eval_accuracy': 0.5034013605442177,
 'eval_runtime': 1.5105,
 'eval_samples_per_second': 194.637,
 'eval_steps_per_second': 6.62,
 'epoch': 20.0}